# TF object detection API in Azure Machine Learning

This notebook demonstrates how to train an object detection model using Tensorflow Object detection API in Azure Machine Learning service




In [ ]:
%load_ext autoreload
%autoreload 2


import wget
import os

from azureml.core import Workspace, Experiment, VERSION
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.environment import Environment
from azureml.train.estimator import Estimator
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails


SUBSCRIPTION_ID = ""
RESOURCE_GROUP = ""
WORKSPACE_NAME = ""

EXP_NAME = 'running-pets'
CLUSTER_NAME = "gpu-cluster"

OBJ_DETECT_URL = "http://storage.googleapis.com/download.tensorflow.org/models/object_detection"
COCO_TAR_BALL = "faster_rcnn_resnet101_coco_11_06_2017.tar.gz"

TRAIN_DIR = "train"
DATA_DIR="data"

DATASET_URL="https://amlgitsamples.blob.core.windows.net/objectdetection"
DATASET_TAR_BALL="petdataset.tar.gz"


os.makedirs(DATA_DIR,exist_ok=True)

print("SDK version:", VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
ws = Workspace(subscription_id = SUBSCRIPTION_ID, 
               resource_group =RESOURCE_GROUP , 
               workspace_name = WORKSPACE_NAME
              )


exp = Experiment(workspace=ws, name=EXP_NAME)

### Download  COCO-pretrained Model for Transfer Learning

In [ ]:
wget.download(os.path.join(OBJ_DETECT_URL,COCO_TAR_BALL),
              out=os.getcwd()
             )

In [ ]:
!tar -xvf $COCO_TAR_BALL -C data --strip 1

###  Upload dataset and model to blob storage

The Oxford-IIIT Pet dataset available in raw format [here](https://www.robots.ox.ac.uk/~vgg/data/pets/). For convinience, it is provided  with the example on TF record file format

In [ ]:
wget.download(os.path.join(DATASET_URL,DATASET_TAR_BALL),
              out=os.getcwd()
             )

In [ ]:
!tar -xvf $DATASET_TAR_BALL -C data --strip 1

In [ ]:
datastore = ws.get_default_datastore()
ds_reference = datastore.upload('data',
                 target_path='pet_dataset',
                 overwrite=True,
                 show_progress=True)


### Create or attach Azure ML compute to be used for training

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget



found = False
cts = ws.compute_targets
if CLUSTER_NAME in cts and cts[CLUSTER_NAME].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[CLUSTER_NAME]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size =  "STANDARD_NC6",max_nodes = 4)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, CLUSTER_NAME, provisioning_config)

print('Checking cluster status...')
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

The docker file contains the instructions to install the TF Models. It's provided with the example, if you'd like to build your own.

For more information on how to perfrom training with custom images on Azure Machine Learning, you can visit 
https://github.com/Azure/AzureML-Containers/

In [ ]:
conda_env = Environment("running_pet_env")
conda_env.python.user_managed_dependencies = True
conda_env.python.interpreter_path = '/opt/miniconda/bin/python'
conda_env.docker.base_image='datashinobi/tf_models:0.1'
conda_env.docker.enabled=True
conda_env.docker.gpu_support=True

## Create estimator

We use Estimator base class here instead of [Tensorflow Estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn.tensorflow?view=azure-ml-py) since we install Tensorflow in the custom base image.

In [ ]:
script_params = {
    '--dataset-path':ds_reference.as_mount(),
    '--epochs':100
}

est = Estimator(source_directory=TRAIN_DIR,
                 script_params=script_params,
                 compute_target=compute_target,
                 entry_script='train.py',
                 environment_definition=conda_env
               )

## Submit Experiment

In [ ]:
run = exp.submit(est)
RunDetails(run).show()

 ##  Monitoring with Tensorboard
 
**This step requires installing locally Tensorflow**,  for more information on tensorboard intergation in Azure Machine Learning 
https://docs.microsoft.com/bs-latn-ba/azure/machine-learning/service/how-to-monitor-tensorboard



In [ ]:
import tensorflow
from azureml.tensorboard import Tensorboard

tb = Tensorboard([run])
tb.start()